다음 링크는 LSTM을 사용하여 Spam 메시지 분류를 수행한 캐글 노트북 => [Link](https://www.kaggle.com/kredy10/simple-lstm-for-text-classification) <br/>

위 노트북에서 사용한 코드를 참고하여<br/>
캐글 데이터셋인 [Women's E-Commerce Clothing Reviews](https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews) 를 분류해보기
- 분류에 사용될 텍스트 데이터 : **`Review Text`** 열을 사용
- 레이블(label) 데이터 : **`Recommended IND`** 열을 사용

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
%matplotlib inline

In [ ]:
from google.colab import files

file = files.upload()

Saving Womens Clothing E-Commerce Reviews.csv to Womens Clothing E-Commerce Reviews.csv


### 1) 데이터 전처리
    
- 데이터셋을 데이터프레임으로 읽어오기
- 필요없는 열(column)을 삭제하기

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
df = pd.read_csv('Womens Clothing E-Commerce Reviews.csv',delimiter=',',encoding='latin-1')
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [ ]:
dfc = df[['Review Text','Recommended IND']]

In [ ]:
dfc.head(4)

,Review Text,Recommended IND
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1


In [ ]:
dfc['Recommended IND'].value_counts()

1    19314
0     4172
Name: Recommended IND, dtype: int64

In [ ]:
dfc.isnull().sum()

Review Text        845
Recommended IND      0
dtype: int64

In [ ]:
dfc = dfc.dropna()

In [ ]:
dfc.isnull().sum()

Review Text        0
Recommended IND    0
dtype: int64

### 2) 텍스트 분류를 수행해주세요.

- 데이터셋 split시 test_size의 비율은 20%로, `random_state = 42` 로 설정 
- Tokenizer의 `num_words=3000` 으로 설정
- pad_sequence의 `maxlen=400` 으로 설정
- 학습 시, 파라미터는 `batch_size=128, epochs=10, validation_split=0.2` 로 설정
- EarlyStopping을 적용 파라미터는 `monitor='val_loss',min_delta=0.0001, patience=3` 로 설정
- evaluate 했을 때의 loss와 accuarcy를 [loss, acc] 형태로 입력하기

In [ ]:
X = dfc['Review Text']
Y = dfc['Recommended IND']

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state = 42)

In [ ]:
max_words = 3000
max_len = 400
batch_size=128
epochs=10 
validation_split=0.2
monitor='val_loss'
min_delta=0.0001
patience=3

In [ ]:
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
def RNN():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(max_words,batch_size,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs, outputs=layer)
    return model

In [ ]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 400)]             0         
                                                                 
 embedding (Embedding)       (None, 400, 128)          384000    
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [ ]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001,patience=3)])

Epoch 1/10
114/114 [==============================] - 89s 784ms/step - loss: 0.3489 - accuracy: 0.8566 - val_loss: 0.2993 - val_accuracy: 0.8689
Epoch 2/10
114/114 [==============================] - 87s 763ms/step - loss: 0.2355 - accuracy: 0.9022 - val_loss: 0.4512 - val_accuracy: 0.8098
Epoch 3/10
114/114 [==============================] - 87s 761ms/step - loss: 0.2116 - accuracy: 0.9121 - val_loss: 0.2830 - val_accuracy: 0.8744
Epoch 4/10
114/114 [==============================] - 87s 762ms/step - loss: 0.1938 - accuracy: 0.9226 - val_loss: 0.3034 - val_accuracy: 0.8835
Epoch 5/10
114/114 [==============================] - 86s 759ms/step - loss: 0.1740 - accuracy: 0.9282 - val_loss: 0.3707 - val_accuracy: 0.8830
Epoch 6/10
114/114 [==============================] - 88s 768ms/step - loss: 0.1579 - accuracy: 0.9366 - val_loss: 0.3312 - val_accuracy: 0.8777


In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
accr = model.evaluate(test_sequences_matrix,Y_test)

142/142 [==============================] - 9s 60ms/step - loss: 0.2964 - accuracy: 0.8817


In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.296
  Accuracy: 0.882
